In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time


In [ ]:

def get_title(soup):
    title_tag = soup.find("span", attrs={'id': 'productTitle'})
    if title_tag:
        return title_tag.get_text(strip=True)
    return ""

def get_price(soup):
    price_tag = soup.find("span", attrs={'id': 'priceblock_ourprice'}) or soup.find("span", attrs={'id': 'priceblock_dealprice'})
    if price_tag:
        return price_tag.get_text(strip=True)
    return ""

def get_rating(soup):
    rating_tag = soup.find("span", attrs={'class': 'a-icon-alt'})
    if rating_tag:
        return rating_tag.get_text(strip=True)
    return ""

def get_review_count(soup):
    review_tag = soup.find("span", attrs={'id': 'acrCustomerReviewText'})
    if review_tag:
        return review_tag.get_text(strip=True)
    return ""

def get_availability(soup):
    availability_tag = soup.find("div", attrs={'id': 'availability'})
    if availability_tag:
        return availability_tag.get_text(strip=True)
    return ""

if __name__ == '__main__':
    HEADERS = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept-Language': 'en-US, en;q=0.5'
    }

    URL = "https://www.amazon.com/s?k=playstation+4&ref=nb_sb_noss_2"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class': 'a-link-normal s-no-outline'})

    # Store the links
    links_list = [link.get('href') for link in links]

    d = {"title": [], "price": [], "rating": [], "reviews": [], "availability": []}

    for link in links_list:
        try:
            new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)
            new_soup = BeautifulSoup(new_webpage.content, "html.parser")

            d['title'].append(get_title(new_soup))
            d['price'].append(get_price(new_soup))
            d['rating'].append(get_rating(new_soup))
            d['reviews'].append(get_review_count(new_soup))
            d['availability'].append(get_availability(new_soup))

            time.sleep(1)  # Be respectful with your request rate

        except Exception as e:
            print(f"An error occurred: {e}")

    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'] = amazon_df['title'].replace('', np.nan)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("Products_1.csv", header=True, index=False)


In [ ]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

In [12]:
amazon_df

,title,price,rating,reviews,availability
0,PlayStation®4 Console – Call of Duty® Modern W...,,4.5 out of 5 stars,116 ratings,In Stock
1,Sony Playstation Classic Console with 20 Plays...,,4.1 out of 5 stars,"2,194 ratings",Only 10 left in stock - order soon.
2,$250 PlayStation Store Gift Card [Digital Code],,4.6 out of 5 stars,"257,519 ratings",Available now
3,Seagate (STGD2000100) Game Drive for PS4 Syste...,,4.7 out of 5 stars,"29,917 ratings",
4,"Wuthur PS4 Controller Wireless 2 Pack, Compati...",,4.3 out of 5 stars,188 ratings,In Stock
5,"PS4 Controller Charger Dock Station, OIVO 1.8H...",,4.7 out of 5 stars,"39,490 ratings",In Stock
6,Star Wars Jedi: Survivor - PlayStation 4,,4.2 out of 5 stars,7 ratings,In Stock
7,"FASIGO PS4 Controller 2 Pack, Wireless PS4 Con...",,4.2 out of 5 stars,470 ratings,In Stock
8,Madden NFL 25 - PlayStation 4,,4.1 out of 5 stars,54 ratings,In Stock
9,Sony Playstation PS4 1TB Black Console,,4.7 out of 5 stars,"1,555 ratings",
